<a href="https://colab.research.google.com/github/chabryl/AI-Redirect-Mapping/blob/main/redirect_monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Benötigte Pakete installieren
!pip install sentence-transformers qdrant-client pandas

import pandas as pd
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
import os
from google.colab import userdata

In [4]:
# 1. Hochladen der CSV-Dateien
from google.colab import files
uploaded = files.upload()

# Dateinamen der hochgeladenen Dateien
crawl_file = 'crawl_all.csv'
error_file = '404_errors.csv'

# 2. CSV-Dateien laden
crawl_data = pd.read_csv(crawl_file)
error_data = pd.read_csv(error_file)

# 3. Word Embeddings nur einmal generieren und speichern
def create_and_save_embeddings(df, save_path, model_name="paraphrase-multilingual-MiniLM-L12-v2"):
    if not os.path.exists(save_path):
        # Lade das vortrainierte Modell
        model = SentenceTransformer(model_name)

        # Erstelle eine Liste der Texte (Title, Meta Description, H1)
        texts = df.apply(lambda row: f"{row['Title']} {row['Meta Description']} {row['H1']}", axis=1).tolist()

        # Generiere Embeddings
        embeddings = model.encode(texts)

        # Speichere die Embeddings und die URLs in eine Datei
        np.savez(save_path, embeddings=embeddings, urls=df['url'].tolist())
        print(f"Embeddings erfolgreich in {save_path} gespeichert.")
    else:
        print(f"Embeddings bereits in {save_path} vorhanden.")

# Erstelle und speichere Embeddings für die crawl_all Datei
embeddings_file = 'crawl_embeddings.npz'
create_and_save_embeddings(crawl_data, embeddings_file, model_name="paraphrase-multilingual-MiniLM-L12-v2")

# 4. Verbindung zur Qdrant-Datenbank
# Hole den API-Key und die API-URL von Qdrant aus den Colab userdata Geheimnissen
api_key_qudrant = userdata.get('api_key_qudrant')
api_url_qudrant = userdata.get('api_url_qudrant')

# Qdrant Client initialisieren
qdrant_client = QdrantClient(url=api_url_qudrant, api_key=api_key_qudrant)

# 5. Embeddings in Qdrant hochladen (falls nicht bereits vorhanden)
def upload_embeddings_to_qdrant(client, collection_name, embeddings_file):
    # Lade die Embeddings und die zugehörigen URLs
    data = np.load(embeddings_file)
    embeddings = data['embeddings']
    urls = data['urls']

    # Überprüfe, ob die Sammlung bereits existiert, und erstelle sie falls nicht
    try:
        client.get_collection(collection_name)
        print(f"Die Sammlung {collection_name} existiert bereits.")
    except:
        print(f"Die Sammlung {collection_name} wird neu erstellt.")
        client.recreate_collection(
            collection_name=collection_name,
            vectors_config=models.VectorParams(
                size=embeddings.shape[1],  # Größe des Embeddings
                distance=models.Distance.COSINE
            )
        )
        client.upload_collection(
            collection_name=collection_name,
            vectors=embeddings,
            payload=[{"url": url} for url in urls]
        )
        print(f"Embeddings erfolgreich in der Sammlung {collection_name} hochgeladen.")

# Qdrant Collection Name
collection_name = "crawl_data_collection"
upload_embeddings_to_qdrant(qdrant_client, collection_name, embeddings_file)

# 6. Ähnlichkeitssuche für 404-Fehler URLs
def find_similar_urls_for_404(client, collection_name, error_data, crawl_data, embeddings_file, model_name="paraphrase-multilingual-MiniLM-L12-v2"):
    # Lade das Modell für die Embedding-Erstellung
    model = SentenceTransformer(model_name)

    # Lade die Embeddings aus der Datei
    data = np.load(embeddings_file)
    embeddings = data['embeddings']
    urls = data['urls']

    for index, row in error_data.iterrows():
        url_404 = row['url']

        # Suche die Daten zur 404-URL in den Crawldaten
        matching_row = crawl_data[crawl_data['url'] == url_404]

        if matching_row.empty:
            print(f"Keine Daten für die 404-URL {url_404} gefunden")
            continue

        # Erstelle Embedding für die 404-Fehlerseite (basierend auf Title, Meta Description, H1)
        text_404 = f"{matching_row['Title'].values[0]} {matching_row['Meta Description'].values[0]} {matching_row['H1'].values[0]}"
        embedding_404 = model.encode([text_404])[0]

        # Suche ähnliche URLs in der Qdrant Datenbank (aber nicht identische URLs)
        results = client.search(
            collection_name=collection_name,
            query_vector=embedding_404,
            limit=5
        )

        # Zeige die Ergebnisse
        print(f"Ähnliche URLs für die 404-Seite {url_404}:")
        for hit in results:
            if hit.payload['url'] != url_404:
                print(f"URL: {hit.payload['url']} (Score: {hit.score})")

# Suche nach ähnlichen URLs für alle 404 Fehler
find_similar_urls_for_404(qdrant_client, collection_name, error_data, crawl_data, embeddings_file, model_name="paraphrase-multilingual-MiniLM-L12-v2")


Saving crawl_all.csv to crawl_all (3).csv
Saving 404_errors.csv to 404_errors (3).csv
Embeddings bereits in crawl_embeddings.npz vorhanden.
Die Sammlung crawl_data_collection existiert bereits.


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Ähnliche URLs für die 404-Seite /de/unternehmen/aktuelles/details/article/abdichtungssystem-fuer-balkone-und-terrassen.html:
URL: /de/bauder-flachdach/produkte/waermedaemmung/terrassensysteme.html (Score: 0.7965164)
URL: /de/bauder-steildach/systeme/passivhaus.html (Score: 0.76800305)
URL: /de/bauder-steilach.html (Score: 0.7602061)
URL: /de/bauder-gruendach/systeme/baudergreen-begehbar.html (Score: 0.757746)
Ähnliche URLs für die 404-Seite /de/jobs-karriere/stellenangebote.html:
URL: /de/service/ihre-ansprechpartner/verarbeitersuche/ergebnisse.html (Score: 0.72123235)
URL: /de/service/ihre-ansprechpartner/verarbeitersuche.html (Score: 0.6416067)
URL: /de/steildach/referenzen/suchergebnisse.html (Score: 0.63271815)
URL: /de/service/seminare-webinare/fuer-architekten-und-planer/webinare/dds-schwammstadt-basis.html (Score: 0.6302194)
Ähnliche URLs für die 404-Seite /de/unternehmen/aktuelles/details/article/diffusionsoffene-wasserdichte-unterdaecher.html:
URL: /de/flachdach/fluessigkunsts